The identify_fast_moving_category function identifies the top n (as specified by no_categories) fast-moving item categories in each department and outlet based on transaction data. 

It first merges the transaction data (trans) with item information (item) on the item_code. Then, it groups the merged data by outlet_code, item_department, and item_category, calculating the total sales quantity for each combination using the sum of total_sales.

A ranking is applied within each outlet and department, with the fastest-moving categories getting the highest ranks based on their total sales quantity. After sorting the data by outlet_code, item_department, and total_sales_qty in descending order, the function filters the top n categories (as defined by no_categories) for each department and outlet, and returns a DataFrame containing this information. 

This provides a clear view of which item categories are performing the best in terms of sales within specific outlets and departments.

In [1]:
%run ./run_script.ipynb

conf = get_conf()

trans = get_datasources(conf)["trans_info"]
item = get_datasources(conf)["item_info"]
stores = get_datasources(conf)["outlets_info"]

trans = pre_process_transaction_info(trans)
item = pre_process_item_info(item)
store = pre_process_stores_info(stores)
no_categories = conf['params']["no_categories"]

In [2]:
def identify_fast_moving_category(trans, item, no_categories):
    """
    Identifying the top 5 fast moving item categories in each department and in each outlet.
    
    Args:
        trans : Pandas DataFrame
            transactions data
        item : Pandas DataFrame 
            items information
        no_categories : number of item categories to be selected
        
    Returns:
        fast_moving_category : Pandas DataFrame
            Contains the top 5 fast moving item categories in each department in each outlet
    """
    
    fast_moving_category = (
        trans
        .merge(item, on="item_code", how="left")
        .groupby(["outlet_code", "item_department", "item_category"])
        .agg(total_sales_qty=("total_sales", "sum"))
        .reset_index()
    )
    
    fast_moving_category["row"] = (
        fast_moving_category
        .groupby(["outlet_code", "item_department"])
        ["total_sales_qty"]
        .rank(method="first", ascending=False)
    )
    
    fast_moving_category = (
        fast_moving_category
        .sort_values(by=["outlet_code", "item_department", "total_sales_qty"], ascending=[True, True, False])
    )
    fast_moving_category = fast_moving_category[fast_moving_category["row"] <= no_categories]
    
    return fast_moving_category


In [3]:
fast_moving_category = identify_fast_moving_category(trans, item, no_categories)
fast_moving_category

,outlet_code,item_department,item_category,total_sales_qty,row
0,A,Beverages,Ambient Liquid Milk,26232,1.0
12,A,Beverages,Powdered Milk,7812,2.0
13,A,Beverages,Single Consumption RTD Beverages,6572,3.0
9,A,Beverages,Multiple Consumption RTD Beverages,3866,4.0
11,A,Beverages,Non Flavored Tea,1761,5.0
...,...,...,...,...,...
271,E,Grocery,Ambient Instant Noodles,5125,1.0
298,E,Grocery,Rice,3836,2.0
306,E,Grocery,Sweet Biscuits & Cookies Regular,2143,3.0
305,E,Grocery,Sugar Confectionary,1508,4.0
